In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

In [2]:
MODEL = "encoder-decoder-t2v"

config = {
    "N_PREV": 60,
    "N_FWD": 30,
    
    "ENCODER_SIZE": 16,
    "DECODER_SIZE": 16,
    "TIME_DIMS": 4,
    
    "LR": 0.001,
    "BATCH_SIZE": 64,
    "EPOCHS": 30,
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: Currently logged in as: kmcguigan (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kiern/.netrc


True

## **Load Data Function**

In [4]:
def get_data(fold):
    with open(f'./data/fold{fold}_data_v2.npy', mode='rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_val = np.load(f)
        y_val = np.load(f)
    # with open(f'./data/fold{fold}_normalizer_v2.pkl', mode='rb') as f:
        # normalizer = pickle.load(f)
    return X_train, y_train, X_val, y_val, None

## **Model Functions**

In [5]:
class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, kernel_size=None, **kwargs):
        self.kernel_size = kernel_size
        super(Time2Vec, self).__init__(**kwargs)
    def get_config(self):
        config = super(Time2Vec, self).get_config()
        config.update({"kernel_size": self.kernel_size})
        return config
    def build(self, input_shape):
        # expects the dims of input to be (batch_size, sequence_length, time_dims)
        # these parameters are for the linear space embedding, not periodic
        # converts time dims to just one, gives one linear time embedding per step in sequence
        self.w = self.add_weight(name='w',
                                 shape=(input_shape[-1], 1),
                                 initializer='uniform',
                                 trainable=True)
        self.p = self.add_weight(name='p',
                                 shape=(input_shape[-2], 1),
                                 initializer='uniform',
                                 trainable=True)
        # These variables are used to be periodic in nature
        # converts the time dim to a vector of length kernel size - 1, adding these together we get vector length k for each time step
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], self.kernel_size-1),
                                 initializer='uniform',
                                 trainable=True)
        self.P = self.add_weight(name='P',
                                 shape=(input_shape[-2], self.kernel_size-1),
                                 initializer='uniform',
                                 trainable=True)
        super(Time2Vec, self).build(input_shape)
    def call(self, inputs):
        original = K.dot(inputs, self.w) + self.p
        sin_trans = K.sin(K.dot(inputs, self.W) + self.P)
        return K.concatenate([original, sin_trans], axis=-1) 

def simple_encoder_decoder():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1))
    x = Time2Vec(config["TIME_DIMS"])(inputs)
    x, hidden_state, cell_state = tf.keras.layers.LSTM(config["ENCODER_SIZE"], 
                                                       return_sequences=True, 
                                                       return_state=True, 
                                                       recurrent_initializer='glorot_uniform', 
                                                       activity_regularizer=L1L2(l1=0.000001, l2=0.000001))(x)
    
    decoder = tf.keras.layers.LSTM(config["DECODER_SIZE"],
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform', 
                                   activity_regularizer=L1L2(l1=0.000001, l2=0.000001))
    dropout = tf.keras.layers.Dropout(0.2)
    decoder_output = tf.keras.layers.Dense(1)
    all_outputs = []
    last_value = tf.expand_dims(inputs[:, -1, 0:1], 1)
    states = [hidden_state, cell_state]
    for i in range(config["N_FWD"]):
        x, hidden_state, cell_state = decoder(last_value, initial_state=states)
        states=[hidden_state, cell_state]
        x = dropout(x)
        x = decoder_output(x)
        last_value = x
        all_outputs.append(x)
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

## **Train Loop**

In [6]:
def train_model(model, X_train, y_train, X_val, y_val):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(X_train, y_train,
              batch_size=config["BATCH_SIZE"],
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=(X_val, y_val),
              validation_batch_size=config["BATCH_SIZE"],
              shuffle=True)
    return model

In [7]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        X_train, y_train, X_val, y_val, _ = get_data(fold)
        model = simple_encoder_decoder()
        model = train_model(model, X_train, y_train, X_val, y_val)
        run.finish()
        del model
        del X_train
        del y_train
        del X_val
        del y_val
        gc.collect()
    return

In [ ]:
train()

Epoch 1/30
2485/2485 [==============================] - 165s 43ms/step - loss: 0.3860 - mae: 0.4777 - val_loss: 0.2423 - val_mae: 0.3759 - lr: 0.0010 - _timestamp: 1648938293.0000 - _runtime: 180.0000
Epoch 2/30
1023/2485 [===========>..................] - ETA: 49s - loss: 0.2534 - mae: 0.3851